In [1]:
import datetime
import pytz

datetime.datetime.utcnow()
# datetime.datetime.utcnow()
# datetime.datetime(2018, 5, 31, 14, 49, 43, 187680)

datetime.datetime.now()
# >>> >>> datetime.datetime.now()
# datetime.datetime(2018, 5, 31, 10, 49, 59, 984947)
#Jak widzimy, mój komputer nie zwraca czasu w formacie UTC, pomimo tego, że w otrzymanym rezultacie nie ma informacji o strefie czasowej
datetime.datetime.now(datetime.timezone.utc)
# >>> datetime.datetime.now(datetime.timezone.utc)
# datetime.datetime(2018, 5, 31, 14, 51, 35, 601355, tzinfo=datetime.timezone.utc)


western = pytz.timezone('US/Pacific')
western.zone
# >>> eastern.zone
# 'US/Pacific'

## API wspiera dwa sposoby tworzenia zmiennych czasowych: z użyciem funkcji 'localize' lub konwersji czasu z innej strefy
# Metoda ustawienia strefy czasowej z użyciem 'localize'
loc_dt = western.localize(datetime.datetime(2018, 5, 15, 12, 34, 0))
# datetime.datetime(2018, 5, 15, 12, 34, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)
# >>>

london_tz = pytz.timezone('Europe/London')
london_dt = loc_dt.astimezone(london_tz)
# >>> london_dt
# datetime.datetime(2018, 5, 15, 20, 34, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)
f = '%Y-%m-%d %H:%M:%S %Z%z'
datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz).strftime(f)
## '2018-05-12 12:15:00 LMT-0001'

## Jak podkreślono w dokumentacji biblioteki pytz, podanie w parametrze tzinfo żądanej strefy czasowej nie zawsze prowadzi o otrzymania pożądanego rezultatu, np. w przypadku czasu londyńskiego
## W dokumentacji zaznaczono, że metoda ta powoduje zwrócenie błędnego czasu w przypadku stref, w których nie dokonuje się zmiany czasu.

#Ogólnie rzecz biorąc, chcemy przechowywać dane w formacie UTC i konwertować je jedynie, gdy wyświetlamy dane użytkownikowi

#Na strefach czasowych można również przeprowadzać obliczenia arytmetyczne
event1 = datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz)
event2 = datetime.datetime(2018, 5, 13, 9, 15, 0, tzinfo = western)
event2 - event1
## Otrzymany rezultat będzie błędny, ponieważ źle ustawiliśmy strefy czasowe 


event1 = london_tz.localize( datetime.datetime(2018, 5, 12, 12, 15, 0))
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0))
event2 - event1



event1 = london_tz.localize((datetime.datetime(2018, 5, 12, 12, 15, 0))).astimezone(datetime.timezone.utc)
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0)).astimezone(datetime.timezone.utc)
event2 - event1

## Zauważ, że w trakcie pracy z arytmetyką stref czasowych, może pojawić się problem nakładania się na siebie daty w dniach, w których następuje zmiana czasu
## musisz wtedy dokonać normalizacji twoich stref

event1 = london_tz.localize( datetime.datetime(2018, 5, 12, 12, 15, 0))
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0))

## przyjrzyj się popularnym strefom czasowym w pytz.common_timezones
pytz.common_timezones
## lub strefom dostępnym w danym kraju
pytz.country_timezones('RU')
# >>> pytz.country_timezones('RU')
# ['Europe/Kaliningrad', 'Europe/Moscow', 'Europe/Simferopol', 'Europe/Volgograd', 'Europe/Kirov', 'Europe/Astrakhan', 'Europe/Saratov', 'Europe/Ulyanovsk', 'Europe/Samara', 'Asia/Yekaterinburg', 'Asia/Omsk', 'Asia/Novosibirsk', 'Asia/Barnaul', 'Asia/Tomsk', 'Asia/Novokuznetsk', 'Asia/Krasnoyarsk', 'Asia/Irkutsk', 'Asia/Chita', 'Asia/Yakutsk', 'Asia/Khandyga', 'Asia/Vladivostok', 'Asia/Ust-Nera', 'Asia/Magadan', 'Asia/Sakhalin', 'Asia/Srednekolymsk', 'Asia/Kamchatka', 'Asia/Anadyr']
# >>>
# >>> pytz.country_timezones('fr')
# ['Europe/Paris']
# >>>


## Strefy czasowe 
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00)).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)
ambig_time_earlier.astimezone(western)
ambig_time.astimezone(western)
ambig_time_later.astimezone(western)
# >>> >>> >>> datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)
# >>> datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)
# >>> datetime.datetime(2002, 10, 27, 2, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)
# >>> >>>
# Zauważ, że dwa ostatnie znaczniki są identyczne!

## W takich przypadkach powinieneś zastosować parametr is_dst w celu wskazania czy obowiązuje czas letni

ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00), is_dst = True).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)
ambig_time_earlier.astimezone(western)
ambig_time.astimezone(western)
ambig_time_later.astimezone(western)
# >> >>> datetime.datetime(2002, 10, 27, 0, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)
# >>> datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)
# >>> datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

## Zauważ, że teraz ten sam czas nie pojawia się dwukrotnie (ale może się pojawić, gdy zechcesz zmierzyć różnice pomiędzy tymi datami a czasem UTC)
loc_dt.strftime(f)



'2018-05-15 12:34:00 PDT-0700'